# SI factor

In [3]:
import numpy as np
import pandas as pd
import datetime
from collections import defaultdict
import glob
import tqdm

In [6]:
import glob
path = '../MarketData_raw/NA/'
files = glob.glob(path + '*.csv')
tickers = list(map(lambda f: f[42:-4], files))
print(' '.join(tickers[:10]))

AN8068571086 BMG491BT1088 BMG6359F1032 BMG812761002 BMG982941046 CH0044328745 CH0048265513 CH0102993182 CH0114405324 GB00B4VLR192


In [7]:
market_data = pd.read_csv('../un_spx_daily_nonadjusted_close.csv', parse_dates=['Date'], index_col='Date')

In [16]:
si_df= market_data.ix[:,0].copy()
for sym in tickers:
    try:
        si = pd.read_csv('../Features/SI_raw/{}.csv'.format(sym), parse_dates=['Date'], index_col='Date')
        si['BS_SH_OUT'].ffill(inplace=True)
        si = si[np.isfinite(si['BS_SH_OUT'])].dropna()
        si = si['SHORT_INT'] / si['BS_SH_OUT'] / 10 ** 6
    except:
        print(pd.read_csv('../Features/SI_raw/{}.csv'.format(sym), parse_dates=['Date'], index_col='Date'))
        si = pd.DataFrame(np.repeat(np.nan, market_data.shape[0]), index=market_data.index)
    
    si_df = pd.concat([si_df, si], axis=1)

Empty DataFrame
Columns: [BS_SH_OUT, SHORT_INT]
Index: []
Empty DataFrame
Columns: [SHORT_INT, BS_SH_OUT]
Index: []


In [18]:
si_df = si_df.drop('AN8068571086', axis=1)

In [19]:
si_df.columns = market_data.columns

In [20]:
for col in si_df.columns:
    try:
#    print()
        si_df[col].loc[:si_df[col].dropna().index[-1]].ffill(inplace=True)
#   break
    except:
        continue

In [21]:
#si_df = si_df.ffill()

In [22]:
si_df = si_df.loc[market_data.index]

In [23]:
si_df = si_df[market_data.columns]

In [32]:
un = si_df.columns[np.isfinite(si_df.loc['2017-04-13']) == True]
si_df[un]= si_df[un].ffill()

In [33]:
si_df.to_csv('../Features/SI_FACTOR.csv')

# DIV YIELD

In [1]:
import glob
path = '../Features/DIV_YIELD/'
files = glob.glob(path + '*.csv')
tickers = list(map(lambda f: f[43:-4], files))
print(' '.join(tickers[:10]))

AN8068571086 BMG491BT1088 BMG6359F1032 BMG812761002 BMG982941046 CH0044328745 CH0048265513 CH0102993182 CH0114405324 GB00B4VLR192


In [4]:
div_yields = pd.DataFrame()
for filename, sym in tqdm.tqdm(zip(files, tickers)):
    mdf = pd.read_csv(filename, index_col=['Date'], parse_dates=['Date'], usecols=['Date', 'EQY_DVD_YLD_12M'])
    div_yields = pd.concat([div_yields, mdf], axis=1) 

768it [05:41,  1.27s/it]


In [ ]:
div_yields.columns = tickers
#div_yields.ix[1:, :].to_csv('../Features/un_spx_div_yields.csv')